In [9]:
import requests
import pymongo
import json
import matplotlib.pyplot as plt
import numpy as np
import math
from datetime import datetime, timezone
from bs4 import BeautifulSoup
import plotly.graph_objects as go
import sys
import pandas as pd

client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')
mydb = client['Upstox']
info = mydb.stocks_url_codes

codes_dict = {}
for i in info.find():
    codes_dict[i['Symbol']] = i['Url']



In [10]:
correct_dates = ['13-Mar-20','23-Mar-20']
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

diwali_date = []
for i in range(1970,2022):
    
    r = requests.get(f'https://diwalidate.co.in/{i}' , headers = headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    price = soup.find('h2' , {'class' : "wow fadeInRight"})
    price.text.split(',')[0]
    diwali_date.append(price.text.split(',')[0])
final_req = []
for d in diwali_date:
    now = d.split(' ')
    to_add = ""
    date = now[0].split('th')[0]
    if(len(date) == 1):
        date = '0' + date
    month = now[1][:3]
    year = now[2][-2:]

    to_add += date + '-' + month + '-' + year
    final_req.append(to_add)
final_req = final_req + correct_dates
    

In [ ]:
final_data = []
i = 0
while(True):
    
    if i==0:
        url = 'https://service.upstox.com/historical/data/NSE_EQ/5900/30?timestamp='
    else:
        url = f'https://service.upstox.com/historical/data/NSE_EQ/5900/30?timestamp={next_timestamp}'
    
    data = json.loads(requests.get(url).text)
    prices = data['data']
    
    
    if(len(prices) == 1):
       
        final_data = prices + final_data
        break
    
    
    if(i>1):
        temp = len(prices)
       
    
        
    next_timestamp = int(prices[0].rpartition('000,')[0])*1000
   
    
    
    i = i+1
    
    
    final_data = prices[1:] + final_data
    
    
for f in final_data:
    if('nan' in f or 'undefined' in f):
        final_data.remove(f)
    else:
        break



req_data = {}
j = 0

a = []


while(j < len(final_data)-1):
    
    if(len(a) == 0):
        
        single = {}
        
        Open = float(final_data[j].split(',')[1])
        high = float(final_data[j].split(',')[2])
        low = float(final_data[j].split(',')[3])
        close = float(final_data[j].split(',')[4])
        time = int(final_data[j].split(',')[0])
        
        single['time'] = datetime.fromtimestamp(float(time)/1000.)
        single['open'] = Open
        single['high'] = high
        single['low'] = low
        single['close'] = close
        
    

        a.append(single)
    
    
    
    epoch_date_currently = int(final_data[j].split(',')[0])
    epoch_date_next = int(final_data[j+1].split(',')[0])
    
    today_date = datetime.fromtimestamp(float(epoch_date_currently)/1000.)
    next_date =  datetime.fromtimestamp(float(epoch_date_next)/1000.)
    
    
    

    
    if(today_date.strftime("%d") == next_date.strftime("%d")):
        
        single = {}
        
        if(final_data[j+1].split(',')[1] != 'undefined' and final_data[j+1].split(',')[1] != 'nan'):
            Open = float(final_data[j+1].split(',')[1])
        else:
            Open = Open
            
        if(final_data[j+1].split(',')[0] != 'undefined' and final_data[j+1].split(',')[0] != 'nan'):
            time = float(final_data[j+1].split(',')[0])
        else:
            time = time
            
        if(final_data[j+1].split(',')[2] != 'undefined' and final_data[j+1].split(',')[2] != 'nan'):
             high = float(final_data[j+1].split(',')[2])
        else:
            high =  high
            
        if(final_data[j+1].split(',')[3] != 'undefined' and final_data[j+1].split(',')[3] != 'nan'):
            low = float(final_data[j+1].split(',')[3])
        else:
            low = low
            
        if(final_data[j+1].split(',')[4] != 'undefined' and  final_data[j+1].split(',')[4] != 'nan'):
            close = float(final_data[j+1].split(',')[4])
        else:
            close = close
            
        
        
        single['time'] = datetime.fromtimestamp(float(time)/1000.)
        single['open'] = Open
        single['high'] = high
        single['low'] = low
        single['close'] = close
        
        a.append(single)

        
    else:
        
        req_data[today_date.strftime("%d-%b-%y")] = a

        if(len(a) < 375):
            print(today_date.strftime("%d : %b : %y"))
            print(len(a))
        a = []

    j = j+1

    

single = {}
Open = float(final_data[j].split(',')[1])
high = float(final_data[j].split(',')[2])
low = float(final_data[j].split(',')[3])
close = float(final_data[j].split(',')[4])
time = int(final_data[j].split(',')[0])

single['time'] = datetime.fromtimestamp(float(time)/1000.)
single['open'] = Open
single['high'] = high
single['low'] = low
single['close'] = close

a.append(single)
req_data[today_date.strftime("%d-%b-%y")] = a
    
len(req_data)



In [16]:
req_data[[*req_data.keys()][-1]].pop(-1)

for key, value in req_data.items():
    if key in final_req:
        continue
    else:
        if(len(value) > 13 or len(value) < 13):
            print(key)


In [17]:
##30 minutes
all_open = []
all_highs = []
all_lows = []
all_closes = []
all_dates = []

for key,val in req_data.items():
    for v in val:
        all_highs.append(v['high'])
        all_lows.append(v['low'])
        all_closes.append(v['close'])
        all_open.append(v['open'])
        all_dates.append(v['time'])

            
            
     

In [ ]:
##2 hours
all_open = []
all_closes = []
all_dates = []
all_highs = []
all_lows = []

for key,val in req_data.items():
    
    date = val[0]['time'].strftime("%d-%b-%y")
    
    if(date in final_req):
        print(date)
        
        if(date == '04-Nov-21' or date == '14-Nov-20' or date == '19-Oct-17'):
        
            temp_highs = []
            temp_lows = []
            
            for i in range(0,len(val)):
                
                temp_highs.append(val[i]['high'])
                temp_lows.append(val[i]['low'])
                
               

                if(i == 0):

                    all_dates.append(val[i]['time'])
                    all_open.append(val[i]['open'])


                else:
                    all_closes.append(val[i]['close'])

            all_highs.append(max(temp_highs))
            all_lows.append(min(temp_lows))
       

        elif(date == '13-Mar-20' or date == '23-Mar-20'):
              
            temp_highs = []
            temp_lows = []
            
            for i in range(0,len(val)):
                
                temp_highs.append(val[i]['high'])
                temp_lows.append(val[i]['low'])
              

                if(i == 2 or i == 6 or i == 10 or i == 11):
                    all_closes.append(val[i]['close'])

                if(i == 0 or i == 3 or i == 7 or i == 11):

                    all_dates.append(val[i]['time'])
                    all_open.append(val[i]['open'])
                    
            
            all_highs.append(max(temp_highs[0:3]))
            all_highs.append(max(temp_highs[3:7]))
            all_highs.append(max(temp_highs[7:11]))
            all_highs.append(max(temp_highs[11:]))


            all_lows.append(min(temp_lows[0:3]))
            all_lows.append(min(temp_lows[3:7]))
            all_lows.append(min(temp_lows[7:11]))
            all_lows.append(min(temp_lows[11:]))

            

              


            
        
        
    else:
        
        temp_highs = []
        temp_lows = []
        
        for i in range(0,len(val)):
            
            temp_highs.append(val[i]['high'])
            temp_lows.append(val[i]['low'])

            if(i == 3 or i == 7 or i == 11 or i == 12):
                all_closes.append(val[i]['close'])

            if(i == 0 or i == 4 or i == 8 or i == 12):

                all_dates.append(val[i]['time'])
                all_open.append(val[i]['open'])
                
                
        all_highs.append(max(temp_highs[0:4]))
        all_highs.append(max(temp_highs[4:8]))
        all_highs.append(max(temp_highs[8:12]))
        all_highs.append(max(temp_highs[12:]))
        
        
        all_lows.append(min(temp_lows[0:4]))
        all_lows.append(min(temp_lows[4:8]))
        all_lows.append(min(temp_lows[8:12]))
        all_lows.append(min(temp_lows[12:]))


                 
     

In [19]:

def er(n):
    
   
    def rolling_diff(t,n):
        x = []
        for i in range(n,len(t)):
            x.append(abs(t[i]-t[i-n]))
        return x
    
    def rolling_sum(t,n):
        x = []
        for i in range(0,len(t)-n+1):
            x.append(sum(t[i:i+n]))
        return x
    def rolling_mean(t,n):
        x = []
        for i in range(0,len(t) - n + 1):
            x.append(sum(t[i:i+n])/n)
        return x

    
    one_day_diff = rolling_diff(all_closes,1)[n-1:]
    fourteen_day_diff = rolling_diff(all_closes,n)
    total_dist = rolling_sum(rolling_diff(all_closes,1),n)
    eff = []
    
    for i in range(0,n):
        eff.append(0)

    

    used_closes = all_closes[n:]
    
    for i in range(0,len(total_dist)):
        if total_dist[i] != 0:
            eff.append(fourteen_day_diff[i]/total_dist[i])
        else:
            eff.append(0)
    return eff

In [ ]:

def er_trades(idx,upper_eff,lower_eff,days,all_trades):
    
    dict_moves = {} 
    
    
    eff = er(days)

    
    perc = []
    length_move = []
    date_begin = []
    move_length = []

    i = 0
    closes_req = all_closes
    dates_req = all_dates
    eff_req = eff

    while(i < len(eff_req)):
        


        if(eff_req[i] <= lower_eff):
            
            trade_info = {}



            high_index = i

            high = -1*sys.maxsize

            low_index = i
            low = sys.maxsize

            eff_index = -1
            eff_max = -1*sys.maxsize

            high_date = ""
            low_date = ""

            j = i-1



            while(j>=0 and eff_req[j]> lower_eff):

                eff_max =max(eff_max,eff_req[j]) 

                if(high < all_highs[j]):

                    high = all_highs[j]
                    high_date = all_dates[j]
                    high_index = j

                if(low > all_lows[j]):

                    low = all_lows[j]
                    low_date = all_dates[j]
                    low_index = j

                j = j-1


            if(high_index - low_index >= 0):    



                while(j>=0 and eff_req[j]< upper_eff):

                    eff_max =max(eff_max,eff_req[j]) 

                    if(high < all_highs[j]):

                        high = all_highs[j]
                        high_date = all_dates[j]
                        high_index = j


                    if(low > all_lows[j]):

                        low = all_lows[j]
                        low_date = all_dates[j]
                        low_index = j


                        '''change it tommorrow'''
                    j = j-1

            else:


                while(j>=0 and eff_req[j]< upper_eff):

                    eff_max =max(eff_max,eff_req[j]) 

                    if(high < all_highs[j]):

                        high = all_highs[j]
                        high_date = all_dates[j]

                        high_index = j


                    if(low > all_lows[j]):

                        low = all_lows[j]
                        low_date = all_dates[j]
                        low_index = j


                        '''change it tommorrow'''
                    j = j-1









            if(j >= 0):

                if(eff_max >= upper_eff and abs(high_index - low_index) >= days and abs(high_index - low_index) <= 50):

                    if(high_index - low_index >= 0): 
                        
                        
                        print("Long")
                        perc.append(((high-low)/low)*100)
                        length_move.append(high_index - low_index)
                        date_begin.append(all_dates[low_index])
                        
                        
                        trade_info['start'] = i 
                        trade_info['high'] = high_index
                        trade_info['low'] = low_index
                        
                        trade_info['move_length'] = high - low
                        trade_info['type'] = "LONG"
                        
                         
                        
                        
                        

                        print("move end date --> ", dates_req[i])

                        print("move start date --> " ,dates_req[low_index])

                        print(high_date,"-->",low_date," High = ",high,"  Low = ",low, "High Ind =" ,high_index, "Low ind= " ,low_index)
                        print(high_index - low_index)



                        print("                  ")
                        print("                  ")
                        print("                  ")
                        print("                  ")
                        print("                  ")
                    else:
                        print("Short")
                        perc.append(((high-low)/high)*100)
                        length_move.append(low_index - high_index)
                        date_begin.append(all_dates[high_index])
                        
                        trade_info['start'] = i 
                        trade_info['high'] = high_index
                        trade_info['low'] = low_index
                        
                        trade_info['move_length'] = high - low
                        trade_info['type'] = "SHORT"
                        
                        

                        print("move end date --> ", dates_req[i])

                        print("move start date --> " ,dates_req[high_index])

                        print(high_date,"-->",low_date," High = ",high,"  Low = ",low, "High Ind =" ,high_index, "Low ind= " ,low_index)
                        print(low_index - high_index)

                        print("                  ")
                        print("                  ")
                        print("                  ")
                        print("                  ")
                        print("                  ")

                        
                    all_trades[idx] = trade_info
                    idx += 1
                while(i < len(eff_req) and eff_req[i] <= upper_eff):

                    i = i+1
            else:
                i = i + 1
        else:
            i = i+1

    dict_moves['perc'] = perc
    dict_moves['duration'] = length_move
    dict_moves['begin'] = date_begin
    
    
    
all_trades= {}
er_trades(0,0.70,0.22,7,all_trades)

In [ ]:
df = pd.DataFrame()

typ = []
s = []
e = []

pr= []
lo = []


begin_index = []

for k,v in all_trades.items():
    begin_index.append(v['start'])
    
idx = 0

profit_count = 0
p = 0
l = 0

for k,v in all_trades.items():
    
    
    if(idx < len(begin_index) - 1):
        
        next_trade_idx = begin_index[idx+1]
    
        if(v['type'] == "LONG"):

            sl = all_highs[v['high']] - v['move_length']/3
            trade_start_price = all_lows[v['start']]
            ongoing_idx = v['start'] + 1

            if(trade_start_price < sl):
                sl = trade_start_price



            while(ongoing_idx < next_trade_idx):

                if(all_highs[ongoing_idx] > (0*v['move_length']) + all_highs[v['high']] and all_highs[ongoing_idx] > trade_start_price):
                    
                   
                    print("Open on  = ",all_dates[v['start']],"-- profit in LONG"," closed on = ",all_dates[ongoing_idx])
                    profit_count += 1
                    p = p + all_closes[ongoing_idx] - trade_start_price
                    
                    typ.append("Long")
                    s.append(all_dates[v['start']])
                    e.append(all_dates[ongoing_idx])
                    pr.append(all_closes[ongoing_idx] - trade_start_price)
                    lo.append(0.000)
                    
                    
                    break
                    
                    
                if(all_lows[ongoing_idx] < sl):
                    
                    
                    print("Open on  = ",all_dates[v['start']],"--loss in LONG = " ,trade_start_price - sl," closed on = ",all_dates[ongoing_idx])
                    l = l + sl - trade_start_price
                    
                    typ.append("Long")
                    s.append(all_dates[v['start']])
                    e.append(all_dates[ongoing_idx])
                    lo.append(sl - trade_start_price)
                    pr.append(0.000)
                    
                    break
                ongoing_idx += 1
                
                
        else:
            
            sl =  all_lows[v['low']] + v['move_length']/3
            trade_start_price = all_highs[v['start']]
            ongoing_idx = v['start'] + 1

            if(trade_start_price > sl):
                sl = trade_start_price



            while(ongoing_idx < next_trade_idx):

                if(all_lows[ongoing_idx] < all_lows[v['low']] - (0*v['move_length'])  and all_lows[ongoing_idx] < trade_start_price):
                    print("Open on  = ",all_dates[v['start']],"-- profit in SHORT"," closed on = ",all_dates[ongoing_idx] )
                    profit_count += 1
                    p = p + trade_start_price - all_closes[ongoing_idx]
                    
                    typ.append("Short")
                    s.append(all_dates[v['start']])
                    e.append(all_dates[ongoing_idx])
                    pr.append(trade_start_price - all_closes[ongoing_idx])
                    lo.append(0.000)

                    break
                if(all_highs[ongoing_idx] > sl):
                    print("Open on  = ",all_dates[v['start']],"--loss is SHORT = ",trade_start_price - sl," closed on = ",all_dates[ongoing_idx])
                    l = l - sl + trade_start_price
                    
                    typ.append("Short")
                    s.append(all_dates[v['start']])
                    e.append(all_dates[ongoing_idx])
                    lo.append(trade_start_price - sl)
                    pr.append(0.000)
                    break
                ongoing_idx += 1
            
            

    idx = idx + 1
    
            

            
    
    
    
    
df['Type'] = typ
df['Trade_start'] = s
df['trade_end'] = e
df['profit'] = pr
df['loss'] = lo
    

long_trades_df = df[df.Type == 'Long']
short_trades_df = df[df.Type == 'Short']